> #### Version:V.1.1 
> #### Git:https://github.com/Zhuang-Zhuang-Liu/Titan-Analysis/tree/main
> #### Power_By:DeepSeek-Chat-V2
> #### Author:Zhuang-Zhuang-Liu
> #### Update_Date:20240623

In [ ]:
!pip install openai==1.35.3
!pip install pyautogen==0.2.31
!pip install 'pyautogen[jupyter-executor]'
!pip install "pyautogen[retrievechat]"

In [ ]:
import shutil
import sys
import json
import autogen
import os
from autogen import Agent,AssistantAgent, UserProxyAgent,ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor,CodeBlock
from autogen.coding.jupyter import JupyterCodeExecutor, LocalJupyterServer
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.capabilities import transform_messages, transforms
from autogen.agentchat.contrib.capabilities.transforms import  MessageHistoryLimiter, MessageTokenLimiter

# git clone
current_directory = os.getcwd()
def clean_library_folder(library_path): 
    try:shutil.rmtree(library_path)
    except OSError as e:print(f"delete {library_path} error: {e}")
clean_library_folder(library_path = current_directory +'/Titan-Analysis')
!git clone https://github.com/Zhuang-Zhuang-Liu/Titan-Analysis.git
sys.path.append( current_directory +'/Titan-Analysis') 

# Titan-Analysis
from utils.utils import show_images_in_directory,folder_clean  # utils
from utils.utils import generate_guide  # rag
from utils.utils import generate_random_data,data_info_put  # IO Data
from agent_zoo.GroupChat import agent_create,agent_da

In [ ]:
# guide rag
guide_path = current_directory +'/Titan-Analysis/rag_zoo/data_analysis_guide.txt'

# agent prompt
prompt_path = current_directory+'/Titan-Analysis/agent_zoo/agent_prompts.json'
with open(prompt_path, "r") as file: agent_prompts = json.load(file)

# llm api
api_zz_deep,url_deep,model_deep = 'sk-1cda68c98daf46a1b931687e698e9ca8','https://api.deepseek.com/v1',"deepseek-chat"
llm_config_deep = {"config_list": [{ "model": model_deep,"base_url": url_deep,"api_key": api_zz_deep,"temperature": 0,"cache_seed":None  }] }  

#folder path
path="coding"
folder_clean(path)

print('***config ready***')

In [ ]:
# task input
print('***请输入数据分析任务，如使用默认任务，请输入“默认”***')
task_info=input()
task_info_default="""{销售收入大于1万、销售成本小于700的用户，主要分布在哪些年龄段和哪些等级城市，先单独分析、然后交叉分析这2个维度
                       备注： 年龄段的划分：18~30岁=青年，30~60=中年，60岁以上=老年}"""
if len(task_info) <= 20:
    task_info = task_info_default
print('[任务已接收]:' + task_info_default)

# data input
print('***请输入需要分析的数据集地址，如使用默认数据集，请输入“默认”***')
data_info = data_info_put(show_data_info=False,data_path=input(),
                          default_data_path=current_directory +'/Titan-Analysis/dataset/Agent_DA_dateset.csv')

agent_da(path=path,llm_config = llm_config_deep, loaded_data=agent_prompts, 
         guide_path=guide_path,task_info=task_info,data_info=data_info)